In [3]:
""" See evernote "ANALYSIS - line5" for details.
Problem in beh eval stroke num count, and in model score. Here figures out the reason
"""

' See evernote "ANALYSIS - line5" for details.\nProblem in beh eval stroke num count, and in model score. Here figures out the reason\n'

In [ ]:

%load_ext autoreload
%autoreload 2

%cd ..

from tools.utils import * 
from tools.plots import *
from tools.analy import *
from tools.calc import *
from tools.analyplot import *
from tools.preprocess import *
from tools.dayanalysis import *

from pythonlib.drawmodel.analysis import *
from pythonlib.tools.stroketools import *

In [ ]:
fd = loadSingleData("Red", "200922", "lines5", 1, load_resaved_data=True, resave_overwrite=False)

### Fore ach trial, print infomration, different methods for extracting stroke numbers. See which trials have problems and mark those 

In [ ]:
## for all trials 

print("trial, len(stroe by peanut) - current stro num -- len(time of raise) - stroke score")
nbad = 0
nall = 0
TARGNUMSTROKE = 2
for t in getIndsTrials(fd):
    if getTrialsFixationSuccess(fd, t):

#         print("Num finger raises (old code):")
        numraise = getTrialsOutcomes(fd, t)["num_finger_raises"][0][0]

        # online tracking
        currentnum = fd["TrialRecord"]["User"]["AdapterParams"][f"{t}"]["bb"]["CurrentStrokeNum"]

        # num peanut raises
        timesOfRaises = getTrialsTimesOfRaises(fd, t)
        num_timesofraise = len(timesOfRaises)

        # num strokes by peanuts
        num_strokes_by_peanut = len(getTrialsStrokesByPeanuts(fd, t))

        # num nans
        touch = getTrialsTouchData(fd, t)
        y = ~np.isnan(touch[:,0])
        num_touch_nans = np.sum(np.diff(y)==1)

        # stroke score
        strok_score = getTrialsOutcomesWrapper(fd, t)["beh_evaluation"]["output"]["numstrokes"]["value"][0][0]


        assert numraise==num_timesofraise
#         assert numraise==num_strokes_by_peanut

        if strok_score != -np.abs(num_strokes_by_peanut-TARGNUMSTROKE):
            tmp = "**"
            nbad +=1
        else:
            tmp = ""
            
        if currentnum!=num_strokes_by_peanut+1:
            tmp2 = "##"
        else:
            tmp2 = ""

#         print([tmp, t, currentnum[0][0], num_timesofraise, num_strokes_by_peanut, num_touch_nans, strok_score])
        print([t, tmp, num_strokes_by_peanut, currentnum[0][0], '--', num_timesofraise, strok_score, tmp2])

        nall+=1

print(f"numbad/all: {nbad}/{nall}")


    

### Once you have trials you want to check, type them in below. See wghat is causes the discrepancy

In [ ]:
%matplotlib inline
# %matplotlib notebook

t=883
plotTrialSingleOverview(fd, t);

print(getTrialsOutcomesWrapper(fd, t))


strokes = getTrialsStrokes(fd, t)

plt.figure(figsize=(15,10))
for i, s in enumerate(strokes):
    plt.plot(s[:,2], i*np.ones_like(s[:,2]), 'o')

peanuts = getTrialsPeanutPos(fd, t)
timesOfRaises = fd["TrialRecord"]["User"]["TrialData"][f"{t}"]["TimesOfRaises"][0]
plt.plot(peanuts[:,2], (i+1)*np.ones_like(peanuts[:,2]), 'xk');
for time in timesOfRaises:
    plt.axvline(time/1000)
    
    
touch = getTrialsTouchData(fd, t)
y = ~np.isnan(touch[:,0])
plt.figure()
plt.plot(y)
plt.title("when touch data not nan")
plt.xlabel("sample num")

plt.figure()
plt.plot(touch[:,2], touch[:,1], 'ob');


print("Num finger raises (old code):")
print(getTrialsOutcomes(fd, t)["num_finger_raises"][0][0])

In [2]:
## SUMMARY:

- current_strok_num (online) is same as offline processed nums trokes (plus 1). this is consistently true, when false, is generally beucase ignored a stroke online since too short
- Times of Raises is Bad. it is same as num raises (old code), which also counts jumps as raises. For pancho thios is bad, since he touches light, so seems like make jumps. 

- Beh evaluator uses Times of Raises to segment peanut positions, so this expaluisn whay the score (but numstrokes and model scroe) are off.

- For Pancho about 10 -30 % trials were wrong during line5, for Red about 1-10%



SyntaxError: invalid syntax (<ipython-input-2-6407be74c614>, line 3)